In [1]:
import nltk
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist


import string
import re

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import one_hot

from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dense
from keras.layers import Flatten



MOVIE = "/home/youcs/Documents/git/Sentiment_Test/data/Movie_Review/"
TWEET = "/home/youcs/Documents/git/Sentiment_Test/data/Tweet_Sentiments/"
DISASTER_TWEET = "/home/youcs/Documents/git/Sentiment_Test/data/Disaster_Tweets/"

TRAIN_MOVIE = "train.tsv"
TEST_MOVIE = "test.tsv"
TRAIN_TWEET = "train.csv"
TEST_TWEET = "test.csv"

############### Helper functions ###############

def load_data(path,file):
    try:
        df = pd.read_csv(path+file)
    except:
        df = pd.read_table(path+file)
    return df

def tokenize2phrases(phrases):
    tokens = [text_to_word_sequence(x) for x in df.Phrase]
    return tokens

def tokenize2unique(phrases):
    tokens = [text_to_word_sequence(x) for x in df.Phrase]
    tokens = [i for x in tokens for i in x]
    return set(tokens)



def lemmatize_tokenized_phrases(tokenized_phrases):
    lemmatizer = WordNetLemmatizer()
    lemm_token_phrases = []
    for tokens in tokenized_phrases:
        tokenize = []
        for token in pos_tag(tokens):
            pos = 'a'
            if token[1].startswith('VB'):
                pos = 'v'
            elif token[1].startswith('NN'):
                pos = 'n'
            tokenize.append(lemmatizer.lemmatize(token[0], pos))
        lemm_token_phrases.append(tokenize)

def remove_stopwords(tokens):
    return [x for x in tokens if x not in stopwords.words('english')]


'''
0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive
plot count
'''

# sns.set(style='darkgrid')
# ax = sns.countplot(x="Sentiment", data=df)

df = load_data(PATH,TRAIN)
unique_tokens = tokenize2unique(df.Phrase)
tokenized_phrases = tokenize2phrases(df.Phrase)
lemmed_phrases = lemmatize_tokenized_phrases(tokenized_phrases)

Using TensorFlow backend.


In [17]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [2]:
encoded_doc = [one_hot(d, len(unique_tokens)) for d in df.Phrase]

In [8]:
padded_doc = pad_sequences(encoded_doc,maxlen=max(map(len,encoded_doc))+1,padding='post')

In [9]:
len(padded_doc[0])

50

In [10]:
len(padded_doc)

156060

In [12]:
from keras.layers import Activation

In [18]:
X_train = padded_doc
y_train = df.Sentiment

In [ ]:
# from gensim.models import Word2Vec
# w2v = Word2Vec(lemm_token_phrases[:10],workers=8,min_count=1)
# print(w2v)
# print(list(w2v.wv.vocab))
# print(w2v)

In [24]:
model = Sequential()
model.add(Embedding(input_dim=len(padded_doc), output_dim=100, input_length=max(map(len,encoded_doc))+1))
model.add(Conv1D(filters=32, kernel_size=8))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=32))
model.add(Activation('relu'))
model.add(Dense(units=1))
model.add(Activation('relu'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 100)           15606000  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 43, 32)            25632     
_________________________________________________________________
activation_8 (Activation)    (None, 43, 32)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 21, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 672)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                21536     
_________________________________________________________________
activation_9 (Activation)    (None, 32)               

In [27]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
  5120/156060 [..............................] - ETA: 20:29 - loss: -15.7979 - accuracy: 0.0248

KeyboardInterrupt: 